![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `mistralai/mistral-large` to make simple chat conversation and tool calls

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook provides a detailed demonstration of the steps and code required to showcase support for Chat models, including the integration of tools and watsonx.ai models.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The purpose of this notebook is to demonstrate how to use Chat models, e.g. `mistralai/mistral-large` by using tools.


## Table of Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [Work with chat messages](#chat)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependencies

In [ ]:
!pip install -U "langchain_ibm>=0.3,<0.4" | tail -n 1

### Define the WML credentials
Use the code cell below to define the WML credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [2]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your WML API key and hit enter: "),
)

### Define the project ID
You need to provide the project ID to give the Foundation Model the context for the call. If you have a default project ID set in Watson Studio, the notebook obtains that project ID. Otherwise, you need to provide the project ID in the code cell below.

In [3]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your project_id and hit enter: ")

<a id="models"></a>
## Set up the Foundation Model on `watsonx.ai`


Specify the `model_id` of the model you will use for the chat with tools.

In [2]:
model_id = "mistralai/mistral-large"

### Define the model parameters

You might need to adjust model `parameters` depending on the model you use.

In [3]:
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters

TextChatParameters.show()

+-------------------+----------------------------------------+-------------------------+
| PARAMETER         | TYPE                                   | EXAMPLE VALUE           |
+===================+========================================+=========================+
| frequency_penalty | float, NoneType                        | 0.5                     |
+-------------------+----------------------------------------+-------------------------+
| logprobs          | bool, NoneType                         | True                    |
+-------------------+----------------------------------------+-------------------------+
| top_logprobs      | int, NoneType                          | 3                       |
+-------------------+----------------------------------------+-------------------------+
| presence_penalty  | float, NoneType                        | 0.3                     |
+-------------------+----------------------------------------+-------------------------+
| response_format   |

In [6]:
params = TextChatParameters(
    temperature=1
)

### Initialize the model

Initialize the `ModelInference` class with the previously set parameters.

In [7]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id,
    credentials=credentials,
    project_id=project_id,
    params=params
)

<a id="chat"></a>
## Work with chat messages 

#### Work with a simple chat message

In [12]:
messages = [
    {
        "role": "user", 
        "content": "What is 1 + 1"
    }
]

simple_chat_response = model.chat(messages=messages)

In [13]:
print(simple_chat_response["choices"][0]["message"]["content"])

 The sum of 1 + 1 is 2. Here it is:

 1
+1
____
 2


#### Work with a simple chat message using chat_stream

In [14]:
messages = [
    {
        "role": "user", 
        "content": "What IBM mainly does?"
    }
]

simple_chat_stream_response = model.chat_stream(messages=messages)

In [14]:
for chunk in simple_chat_stream_response:
    print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

 IBM (International Business Machines Corporation) is a multinational technology and consulting company that operates in more than 170 countries. Here are some of the main areas that IBM is involved in:

1. **Technology and Consulting Services**: IBM Global Services is the world's largest business and technology services provider. It offers consulting, software development, and IT services.

2. **Cloud Computing**: IBM Cloud offers a wide range of services for public, private, and hybrid cloud environments. It includes Infrastructure as a Service (IaaS), Software as a Service (SaaS), and Platform as a Service (PaaS) offerings.

3. **Artificial Intelligence (AI) / Cognitive Computing**: IBM Watson is a flagship product in this area, offering a range of AI services, including natural language processing, machine learning, and predictive analytics.

4. **Software**: IBM develops, manufactures, and licenses software for various domains. This includes operating systems (like z/OS), middlewa

#### Work with an advanced chat message

In [18]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Who won the world series in 2020?"
            }
        ],
    },
    {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Where was it played?"
            }
        ],
    }
]

advanced_chat_response = model.chat(messages=messages)

In [19]:
print(advanced_chat_response["choices"][0]["message"]["content"])

 The 2020 World Series was played at Globe Life Field in Arlington, Texas. Due to the COVID-19 pandemic, the entire series was played at a neutral site, which was the new home of the Texas Rangers. It was the first time since 1944 that the World Series was played at a single location. The Dodgers defeated the Tampa Bay Rays in six games to win their seventh World Series title.


#### Work with chat messages using `tools` and `tool_choice`

In [15]:
def get_current_weather(location: str, unit: str = "celsius") -> dict:
    """
    Get the current weather in a given location.

    Parameters:
    - location (str): The city and state, e.g., "San Francisco, CA".
    - unit (str): The unit for temperature, either "celsius" or "fahrenheit". Defaults to "celsius".

    Returns:
    - dict: A dictionary containing the current weather details.
    """
    weather_data = {
        "location": location,
        "temperature": 20,
        "unit": unit,
        "description": "Partly cloudy"
    }
    return weather_data

Converting raw python function to correct tool schema

In [16]:
import json
from langchain_ibm.chat_models import convert_to_openai_tool

formatted_tools = [convert_to_openai_tool(get_current_weather)]
print(json.dumps(formatted_tools, indent=4))

[
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location. Parameters:\n- location (str): The city and state, e.g., \"San Francisco, CA\".\n- unit (str): The unit for temperature, either \"celsius\" or \"fahrenheit\". Defaults to \"celsius\".",
            "parameters": {
                "properties": {
                    "location": {
                        "type": "string"
                    },
                    "unit": {
                        "default": "celsius",
                        "type": "string"
                    }
                },
                "required": [
                    "location"
                ],
                "type": "object"
            }
        }
    }
]


In [23]:
messages = [
    {"role": "user", "content": "What's the weather like in Boston today?"},
]

tool_choice = {"type": "function", "function": {"name": "get_current_weather"}}

tool_response = model.chat(messages=messages, tools=formatted_tools, tool_choice=tool_choice)

In [24]:
print(json.dumps(tool_response["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "content": "",
    "tool_calls": [
        {
            "id": "chatcmpl-tool-87eea10be7fd48f386c6f4602247cbc4",
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "arguments": "{\"location\":\"Boston,MA\"}"
            }
        }
    ]
}


#### Work with chat messages using `tools` and `tool_choice_option`

In [25]:
def add_numbers(a: float, b: float) -> float:
    """
    Adds two numbers.

    Parameters:
    - a (float): The first number.
    - b (float): The second number.

    Returns:
    - float: The result of a + b.
    """
    return a + b


def multiply_numbers(a: float, b: float) -> float:
    """
    Multiplies two numbers.

    Parameters:
    - a (float): The first number.
    - b (float): The second number.

    Returns:
    - float: The result of a * b.
    """
    return a * b

Converting raw python function to correct tool schema

In [29]:
tool_choice_option = "auto"

formatted_tools = [convert_to_openai_tool(tool) for tool in [add_numbers, multiply_numbers]]

print(json.dumps(formatted_tools, indent=4))

[
    {
        "type": "function",
        "function": {
            "name": "add_numbers",
            "description": "Adds two numbers. Parameters:\n- a (float): The first number.\n- b (float): The second number.",
            "parameters": {
                "properties": {
                    "a": {
                        "type": "number"
                    },
                    "b": {
                        "type": "number"
                    }
                },
                "required": [
                    "a",
                    "b"
                ],
                "type": "object"
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "multiply_numbers",
            "description": "Multiplies two numbers. Parameters:\n- a (float): The first number.\n- b (float): The second number.",
            "parameters": {
                "properties": {
                    "a": {
                        "type": "number"
     

In [30]:
messages = [
    {"role": "user", "content": "What is 5 + 6?"},
]

tools_response = model.chat(messages=messages, tools=formatted_tools, tool_choice_option=tool_choice_option)

print(json.dumps(tools_response["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "tool_calls": [
        {
            "id": "vfrqLLJ8m",
            "type": "function",
            "function": {
                "name": "add_numbers",
                "arguments": "{\"a\": 5, \"b\": 6}"
            }
        }
    ]
}


In [31]:
messages = [
    {"role": "user", "content": "What is 5 * 6?"},
]

tools_response_2 = model.chat(messages=messages, tools=formatted_tools, tool_choice_option=tool_choice_option)

print(json.dumps(tools_response_2["choices"][0]["message"], indent=4))

{
    "role": "assistant",
    "tool_calls": [
        {
            "id": "nPLAOHLZs",
            "type": "function",
            "function": {
                "name": "multiply_numbers",
                "arguments": "{\"a\": 5, \"b\": 6}"
            }
        }
    ]
}


#### Execute tool calls

We organize the two functions into a dictionary where keys represent the function name, and values are the function.

In [32]:
names_to_functions = {
    "add_numbers": add_numbers,
    "multiply_numbers": multiply_numbers,
}

In [33]:
tool_call = tools_response["choices"][0]["message"]["tool_calls"]
function_name = tool_call[0]["function"]["name"]
function_params = json.loads(tool_call[0]["function"]["arguments"])
print(f"Executing function: `{function_name}`, with parameters: {function_params}")

function_result = names_to_functions[function_name](**function_params)
print(f"Function result: {function_result}")

Executing function: `add_numbers`, with parameters: {'a': 5, 'b': 6}
Function result: 11


In [34]:
tool_call = tools_response_2["choices"][0]["message"]["tool_calls"]
function_name = tool_call[0]["function"]["name"]
function_params = json.loads(tool_call[0]["function"]["arguments"])
print(f"Executing function: `{function_name}`, with parameters: {function_params}")


function_result = names_to_functions[function_name](**function_params)
print(f"Function result: {function_result}")

Executing function: `multiply_numbers`, with parameters: {'a': 5, 'b': 6}
Function result: 30


<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to work with chat models using tools and watsonx.ai.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.